In [1]:
## Libraries
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import pv_model_class
import pandas as pd

In [2]:
# Constants and variables

Iscn = 8.21 # nominal short circuit current
Vocn = 32.9 # nominal open circuit voltage
Imp = 7.61 # array current at MPP
Vmp = 26.3 # array voltage at MPP
Pmax_e = Vmp * Imp # experimental array max power output
Kv = -0.123 # voltage temperature coefficient
Ki = 0.0032 # current temperature coefficient
Ns = 54 # number of series cells
Gn = 1000. # nominal irradiance
G = Gn
Tn = 25. + 273.15 # nominal operating temperature
T = Tn

Egap = 1.8 * 10 ** -19 # Bandgap of silicon (silicio cristalino)

err = 0.0001
inverter = 0.95
array_dim = [1, 1]

In [4]:
df = pd.read_csv('/Users/margotadam/Downloads/43386d97fd77fc514fa97f6166756b56/242127_47.69_-121.10_2015.csv')


In [7]:
new_header = df.iloc[1]
tmy = df[2:]
tmy.columns = new_header
tmy = tmy.dropna(axis=1)
tmy['Hour'][30]

'14'

In [8]:
tmy_july = tmy[(tmy['Month'] == '7') & (tmy['Day'] == '3')]
tmy_jan = tmy[(tmy['Month'] == '1') & (tmy['Day'] == '2')]


In [11]:
### Equations from Masters textbook (table 4.1 and table 4.2)
DNI = int(tmy['DNI'][12795])
DHI = int(tmy['DHI'][12795])
GHI = int(tmy['GHI'][12795])

# ground reflectance
rho = 0.2
# number of days
#month = int(tmy['Month'][12795])
#day = int(tmy['Day'][12795])
#n = (30*month - 30) + day

# calculated by hand for 9/24
n = 267
# hour
hour = int(tmy['Hour'][12795])
# latitude
lat = 47.69
# collector azimuth
az_c = 0
# collector tilt angle
sigma = lat * 0.76 + 3.1
# declination
decl = 23.45 * np.sin(360/365 * (n - 81))
# hour angle
H = 15 * (12 - hour)
# solar altitude
beta = np.arcsin(np.cos(lat) * np.cos(decl) * np.cos(H) + np.sin(lat) * np.sin(decl))
# solar azimuth
az_s = np.arcsin(np.cos(decl) * np.sin(H) / np.cos(beta))

cos_theta = np.cos(beta) * np.cos(az_s - az_c) * np.sin(sigma) + np.sin(beta) * np.cos(sigma)
B = DNI * cos_theta
D = DHI * (1 + np.cos(sigma)) / 2
R = GHI * rho * (1 - np.cos(sigma)) / 2
G = B + D + R
print(G)

250.4324849782308


In [9]:
## calculating PV model for consecutive hours throughout a day

pv_mod = pv_model_class.PvModel(Iscn, Vocn, Imp, Vmp, Kv, Ki, Ns, Gn, G, Tn, T, Egap, err, inverter, array_dim)
# find resistor values and diode ideality constant
Rs, Rp, a = pv_mod.find_resistors()
# find voltage, current, and power arrays
v, i, P = pv_mod.calculate_power(Rs, Rp, a)

max_power = []

# calculate day number
#    7/3 -- 184
n = 2
# ground reflectance
rho = 0.2
# latitude
lat = 47.69
# collector azimuth
az_c = 0
# collector tilt angle
sigma = lat * 0.76 + 3.1

for i in range(2, 24*2 + 2, 2):
    # shift to get to month of July 3rd
    # i += 8786
    
    # shift to get to Jan 2nd
    i += 50
    hour = int(tmy['Hour'][i])
    
    DNI = int(tmy['DNI'][i])
    DHI = int(tmy['DHI'][i])
    GHI = int(tmy['GHI'][i])
    pv_mod.G = pv_mod.find_irradiance(DNI, DHI, GHI, rho, n, hour, lat, az_c, sigma)
    
    v, i, P = pv_mod.calculate_power(Rs, Rp, a)
    max_idx = np.argmax(P)
    if P[max_idx] < 0.001:
        P[max_idx] = 0
    max_power.append(P[max_idx])

print(max_power)

## if pv array of 3 strings with 2 modules in series per string (6 total)
max_power_array = max_power * 6

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.036874374112418534, 18.874678324745716, 15.690924739526677, 72.0408116258169, 24.73208104704921, 16.538375698519136, 26.69471806669538, 2.0599573028765286, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
